## 0. 전반적인 Process
- 현황
    - 샘플 데이터를 활용한 Prompt Engineering 수행 중
- 과제
    - Postgre DB 연계 필요
    - BPI 연계 필요

In [ ]:
from IPython.display import Image, display
import base64

def mm(graph: str) -> None:
    """
    Mermaid 그래프를 이미지로 렌더링하여 표시합니다.

    이 함수는 Mermaid 문법으로 작성된 그래프 문자열을 받아
    base64로 인코딩한 후, Mermaid 온라인 렌더링 서비스를 통해
    이미지로 변환하여 Jupyter Notebook이나 유사한 환경에서 표시합니다.

    Args:
        graph (str): Mermaid 문법으로 작성된 그래프 문자열

    Returns:
        None: 이미지를 직접 표시하므로 반환값은 없습니다.

    Example:
        mm('''
        graph TD
        A[Start] --> B{Is it?}
        B -- Yes --> C[OK]
        B -- No --> D[End]
        ''')
    """
    # 그래프 문자열을 UTF-8로 인코딩
    graphbytes = graph.encode("utf-8")
    
    # 인코딩된 바이트를 base64로 변환
    base64_bytes = base64.b64encode(graphbytes)
    
    # base64 바이트를 ASCII 문자열로 디코딩
    base64_string = base64_bytes.decode("ascii")
    
    # Mermaid 온라인 렌더링 서비스 URL과 base64 문자열을 결합하여 이미지 표시
    display(Image(url="https://mermaid.ink/img/" + base64_string))

mm("""
graph LR
    A[API Key 입력] --> B[PostGre DB 연계]
    B --> C[BPI 계산]
    C --> D[강약점 선택]
    D --> E[강약점 데이터전처리]
    E --> F[Prompt Engineering]
    F --> G[완료]
""")

## 1. 필수 라이브러리 임포트 및 환경 설정
- .env 파일에 '자신' 또는 '더존' api key 저장: 
    * OPENAI_API_KEY=sk-proj-.....

In [34]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableParallel, RunnableMap, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from typing import Dict, Any, Literal
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackHandler
from langchain.cache import SQLiteCache
from operator import itemgetter
from tabulate import tabulate


# env 파일 읽기
load_dotenv()

# OpenAI API 키 가져오기
openai_api_key = os.getenv("OPENAI_API_KEY")

## 2. Sample Data 생성
- 데이터 출력 방식은 협의 필요 (\w 조희성 과장님, 이도은 대리님)
- 현재 출력물
    - df_company_info : 회사명, 업종, 재무/회계 데이터
    - df_partner_info : df_company_info의 회사의 매입/매출 거래처 정보 등
- 향후 : Postgre DB와 연계

In [ ]:
# 재현 가능성을 위해 난수 생성기의 시드를 설정합니다.
np.random.seed(42)

# 회사 및 업종 정보 생성
num_companies = 100  # 총 100개의 회사 생성
company_names = [f'Company_{i+1}' for i in range(num_companies)]  # 회사명 리스트 생성 (Company_1, Company_2, ..., Company_100)
industries = ['Manufacturing', 'Retail', 'Technology', 'Healthcare', 'Finance']  # 업종 리스트 정의

# 각 회사에 대한 고정된 업종 리스트 생성
# 각 회사의 업종을 무작위로 할당
company_industries = [np.random.choice(industries) for _ in range(num_companies)]  

# 날짜 생성 (2020-01 ~ 2024-09)
# 2020년 1월부터 2024년 9월까지의 월별 날짜 리스트 생성
dates = pd.date_range(start="2020-01-01", end="2024-09-01", freq='MS').strftime("%Y-%m").tolist()  

# 기본 정보 데이터 생성 함수 정의
def generate_basic_data(company, industry, date):
    """
    각 회사의 특정 날짜에 대한 기본 재무 데이터를 생성하는 함수입니다.
    
    Args:
        company (str): 회사명
        industry (str): 업종명
        date (str): 날짜 (YYYY-MM 형식)
    
    Returns:
        dict: 생성된 회사의 기본 재무 정보
    """
    # 각 재무 항목의 값을 임의로 생성하여 반환합니다.
    base_asset_value = 30000  # 기본 총자산 값 설정 (3억원)
    total_assets = round(base_asset_value * (1 + np.random.normal(0, 0.05)), 0)  # 기본 총자산 값에서 5% 변동 (정규분포 이용, 소수점 없이)
    revenue = round(np.random.uniform(1000, 10000), 0)  # 매출액은 1천만원 ~ 1억원 사이의 값으로 설정 (소수점 없이)
    operating_profit = round(revenue * np.random.uniform(-0.2, 0.2), 0)  # 영업이익은 매출액의 -20%~20% 범위로 설정 (소수점 없이)
    net_income = round(operating_profit * np.random.uniform(-0.5, 0.8), 0)  # 당기순이익은 영업이익의 -50%~80% 범위로 설정 (소수점 없이)
    short_term_loans = round(np.random.uniform(500, 5000), 0)  # 단기차입금은 500만원 ~ 5000만원 범위로 설정 (소수점 없이)
    long_term_loans = round(np.random.uniform(1000, 10000), 0)  # 장기차입금은 1000만원 ~ 1억원 범위로 설정 (소수점 없이)
    total_loans = short_term_loans + long_term_loans  # 총 차입금 계산
    loan_to_sales = round((total_loans / revenue) * 100, 2)  # 매출대비차입금 비율을 백분율로 계산
    working_capital_turnover = round(np.random.uniform(1, 5), 2)  # 운전자금회전율은 1~5회 사이의 값으로 설정
    operating_cash_flow = round(np.random.uniform(500, 5000), 0)  # 영업활동현금흐름은 500만원 ~ 5000만원 범위로 설정 (소수점 없이)
    net_cash_flow = round(operating_cash_flow - total_loans, 0)  # 순현금흐름 = 영업활동현금흐름 - 총 차입금 (소수점 없이)
    ar_balance = round(np.random.uniform(1000, 5000), 0)  # 월별 매출채권 규모 (1000만원 ~ 5000만원, 소수점 없이)
    ar_collection_period = round(np.random.uniform(30, 90), 0)  # 월별 매출채권 회수기일 (30일 ~ 90일, 소수점 없이)
    ap_balance = round(np.random.uniform(800, 4000), 0)  # 월별 매입채무 규모 (800만원 ~ 4000만원, 소수점 없이)
    ap_payment_period = round(np.random.uniform(20, 80), 0)  # 월별 매입채무 회수기일 (20일 ~ 80일, 소수점 없이)
    
    return {
        '기업명': company,
        '업종': industry,
        '날짜': date,
        # 성장성 지표
        '매출액증가율': round(np.random.uniform(5, 15), 2),  # 매출액증가율 (5% ~ 15% 사이)
        '총자산증가율': round(np.random.uniform(3, 10), 2),  # 총자산증가율 (3% ~ 10% 사이)
        '총자산': int(total_assets),  # 총자산 (정수형)
        '매출액': int(revenue),  # 매출액 (정수형)
        # 수익성 지표
        '영업이익': int(operating_profit),  # 영업이익 (정수형)
        '영업이익률': round(operating_profit / revenue * 100, 2) if revenue != 0 else round(np.random.uniform(-20, 20), 2),  # 영업이익률 (백분율)
        '당기순이익': int(net_income),  # 당기순이익 (정수형)
        '당기순이익률': round(net_income / revenue * 100, 2) if revenue != 0 else round(np.random.uniform(-30, 30), 2),  # 당기순이익률 (백분율)
        # 재무 안정성 지표
        '단기차입금': int(short_term_loans),  # 단기차입금 (정수형)
        '장기차입금': int(long_term_loans),  # 장기차입금 (정수형)
        '매출대비차입금': loan_to_sales,  # 매출대비차입금 비율 (백분율)
        '운전자금회전율': working_capital_turnover,  # 운전자금회전율
        # 인적 관리 지표
        '인원수': np.random.randint(50, 300),  # 임직원 수 (50 ~ 300명 사이)
        '월평균급여액': round(np.random.uniform(200, 500), 1),  # 월평균 급여액 (200 ~ 500만원)
        '월매출창출액': round(np.random.uniform(5000, 20000), 1),  # 월 매출 창출액 (5000 ~ 20000만원)
        # 현금흐름 지표
        '영업활동현금흐름/매출액': round(np.random.uniform(0.05, 0.2), 2),  # 영업활동현금흐름/매출액 비율 (5% ~ 20%)
        '영업활동현금흐름': int(operating_cash_flow),  # 영업활동현금흐름 (정수형)
        '순현금흐름': int(net_cash_flow),  # 순현금흐름 (정수형)
        # 거래처 안정성 지표
        '월별_매출채권_규모': int(ar_balance),  # 매출채권 규모 (정수형)
        '월별_매출채권_회수기일': int(ar_collection_period),  # 매출채권 회수기일 (정수형)
        '월별_매입채무_규모': int(ap_balance),  # 매입채무 규모 (정수형)
        '월별_매입채무_회수기일': int(ap_payment_period),  # 매입채무 회수기일 (정수형)
    }

# 기본 정보 데이터 생성
# 각 회사와 날짜별로 기본 재무 데이터를 생성하여 리스트로 저장
basic_data = [
    generate_basic_data(company, industry, date)
    for company, industry in zip(company_names, company_industries)
    for date in dates
]

# 기본 정보 데이터프레임 생성
df_company_info = pd.DataFrame(basic_data)  # 회사 기본 정보로 데이터프레임 생성

# 데이터프레임 출력
print(df_company_info)

# 거래처 정보 데이터 생성 함수 정의
def generate_partner_data(company, date):
    """
    각 회사의 특정 날짜에 대해 상위 매출처 및 매입처 정보를 생성하는 함수입니다.
    
    Args:
        company (str): 회사명
        date (str): 날짜 (YYYY-MM 형식)
    
    Returns:
        list of dict: 생성된 매출처 및 매입처 정보 리스트
    """
    # 상위 5개 매출처 및 매입처 정보 무작위로 생성
    sales_companies = list(np.random.choice(sales_partner_pool, 5, replace=False))
    purchase_companies = list(np.random.choice(purchase_partner_pool, 5, replace=False))

    # 각 매출처 및 매입처의 신용등급을 무작위로 선택
    sales_grades = list(np.random.choice(['AAA', 'AA+', 'AA', 'A+', 'A', 'BBB+', 'BBB', 'BB', 'B', 'CCC-'], 5, replace=False))
    purchase_grades = list(np.random.choice(['AAA', 'AA+', 'AA', 'A+', 'A', 'BBB+', 'BBB', 'BB', 'B', 'CCC-'], 5, replace=False))

    # 매출처 및 매입처의 집중도를 Dirichlet 분포를 이용해 생성하고, 총합이 100%가 되도록 정규화
    sales_concentrations = list(np.round(np.random.dirichlet(np.ones(5), 1)[0] * 100, 2))
    sales_concentrations = list(np.round(np.array(sales_concentrations) / sum(sales_concentrations) * 100, 2))

    purchase_concentrations = list(np.round(np.random.dirichlet(np.ones(5), 1)[0] * 100, 2))
    purchase_concentrations = list(np.round(np.array(purchase_concentrations) / sum(purchase_concentrations) * 100, 2))

    # 매출처 및 매입처 정보 반환 (각 회사 당 5개)
    return [
        {
            '기업명': company,
            '날짜': date,
            '매출처_회사명': sales_companies[i],
            '매출처_신용등급': sales_grades[i],
            '매출처_집중도': sales_concentrations[i],
            '매입처_회사명': purchase_companies[i],
            '매입처_신용등급': purchase_grades[i],
            '매입처_집중도': purchase_concentrations[i]
        }
        for i in range(5)
    ]

# 회사명 리스트에서 중복을 최소화하며 선택할 수 있도록 셋 생성
sales_partner_pool = list(set(f'SalesPartner_{i+1}' for i in range(200)))  # 매출처 Pool 생성 (200개)
purchase_partner_pool = list(set(f'PurchasePartner_{i+1}' for i in range(200)))  # 매입처 Pool 생성 (200개)

# 거래처 정보 데이터 생성
# 각 회사와 날짜별로 거래처 정보를 생성하여 리스트로 저장
partner_data = [
    partner
    for company in company_names
    for date in dates
    for partner in generate_partner_data(company, date)
]

# 거래처 정보 데이터프레임 생성
df_partner_info = pd.DataFrame(partner_data)  # 거래처 정보로 데이터프레임 생성

# 샘플 출력
print(df_company_info.head())  # 회사 기본 정보 샘플 출력
print(df_partner_info.head())  # 거래처 정보 샘플 출력

- Company_1에 대한 샘플 데이터

In [ ]:
from pprint import pprint

print('#'*10, 'Company_1에 대한 정보', '#'*10)
pprint(df_company_info[df_company_info['기업명']== 'Company_1'])

print('\n\n', '#'*10, 'Company_1의 매출/매입 거래처 정보', '#'*10)
pprint(df_partner_info[df_partner_info['기업명']== 'Company_1'])

## 3. 데이터전처리
- 현황 : 성장성, 수익성 구현
- 전처리 함수
    - preprocess_growth_data : '총자산', '매출액' 등 성장성 관련 Data를 Dict 형태로 변경
    - preprocess_profitability_data : '영업이익', '당기순이익' 등 수익성 관련 Data를 Dict 형태로 변경

In [ ]:
# 재현성을 위한 Random seed 설정
np.random.seed(42)

def preprocess_growth_data(df_company_info):
    """
    성장성 지표 데이터를 전처리하여 필요한 JSON 형식의 구조로 변환하는 함수.
    
    Args:
        df_company_info (DataFrame): 회사 재무 정보 데이터프레임
    
    Returns:
        dict: 전처리된 성장성 지표 데이터 (JSON 형태)
    """
    # 최신 년월 추출: 데이터프레임에서 가장 최신의 '날짜' 값을 가져옴
    latest_year_month = df_company_info['날짜'].max()
    latest_year = int(latest_year_month.split('-')[0])  # 최신 년도 추출
    latest_month = int(latest_year_month.split('-')[1])  # 최신 월 추출

    # 결과를 저장할 데이터 딕셔너리 정의
    growth_data = {
        'latest_year_month': latest_year_month,  # 최신 년월 기록
        'year_level_data': {},  # 연간 수준 데이터
        'year_rate_data': {},  # 연간 증가율 데이터
        'recent_data': {}  # 최근 12개월 데이터
    }

    # 'year_level_data' 및 'year_rate_data'에 대해 과거 데이터를 추출
    num_years = 3  # 추출할 과거 연도의 수, 필요에 따라 변경 가능
    for year_offset in range(1, num_years + 1):
        target_year = latest_year - year_offset  # 추출할 연도의 년도 값
        year_key = f"{target_year}년"  # 연도별 키 생성
        december_date = f"{target_year}-12"  # 12월 데이터 날짜 포맷 생성
        
        # 해당 연도의 12월 데이터를 필터링
        december_data = df_company_info[df_company_info['날짜'] == december_date]
        # 해당 연도에 대한 전체 월 데이터를 필터링
        annual_data = df_company_info[df_company_info['날짜'].str.startswith(f"{target_year}")]
        
        # 12월 데이터와 연간 데이터가 모두 있는 경우 처리
        if not december_data.empty and not annual_data.empty:
            # 총자산: 해당 연도의 12월 기준 데이터 사용
            total_assets = int(round(december_data.at[december_data.index[0], '총자산']))
            # 매출액: 해당 연도 전체 매출액의 합계 계산
            revenue = int(round(annual_data['매출액'].sum()))
            
            # 총자산증가율 계산: 이전 연도의 데이터가 존재하는 경우
            prev_year_december_date = f"{target_year - 1}-12"
            prev_december_data = df_company_info[df_company_info['날짜'] == prev_year_december_date]
            if not prev_december_data.empty:
                prev_total_assets = prev_december_data.at[prev_december_data.index[0], '총자산']
                # 전년도 총자산과 비교하여 증가율을 계산
                asset_growth_rate = round(((total_assets - prev_total_assets) / prev_total_assets) * 100, 2)
            else:
                asset_growth_rate = 0

            # 매출액증가율 계산: 이전 연도의 데이터가 존재하는 경우
            prev_annual_data = df_company_info[df_company_info['날짜'].str.startswith(f"{target_year - 1}")]
            if not prev_annual_data.empty:
                prev_revenue = prev_annual_data['매출액'].sum()
                if prev_revenue != 0:
                    # 전년도 매출액과 비교하여 증가율을 계산
                    revenue_growth_rate = round(((revenue - prev_revenue) / prev_revenue) * 100, 2)
                else:
                    revenue_growth_rate = 0  # 이전 연도의 매출액이 0인 경우 증가율을 0으로 설정
            else:
                revenue_growth_rate = 0
            
            # 연도별 데이터 딕셔너리에 추가
            growth_data['year_level_data'][year_key] = {
                '총자산': total_assets,
                '매출액': revenue
            }
            growth_data['year_rate_data'][year_key] = {
                '총자산증가율': asset_growth_rate,
                '매출액증가율': revenue_growth_rate
            }
        else:
            # 데이터가 없는 경우 기본 값 (0) 추가
            growth_data['year_level_data'][year_key] = {
                '총자산': 0,
                '매출액': 0
            }
            growth_data['year_rate_data'][year_key] = {
                '총자산증가율': 0,
                '매출액증가율': 0
            }

    # 최신 년월의 데이터 추가 ('연말 예상' 데이터)
    estimate_key = f"{latest_year}년(E)"  # '연말 예상' 키 설정
    # 직전 월의 데이터가 없는 경우 존재하는 가장 최근의 데이터로 이동
    max_iterations = 12  # 최대 반복 횟수 설정
    iteration_count = 0
    previous_year_month = pd.to_datetime(latest_year_month) - pd.DateOffset(months=1)
    # 직전 월 데이터를 찾기 위해 반복적으로 이전 달로 이동
    while previous_year_month.strftime("%Y-%m") not in df_company_info['날짜'].values:
        previous_year_month -= pd.DateOffset(months=1)
        iteration_count += 1
        if iteration_count >= max_iterations:
            raise ValueError("Exceeded maximum iterations while searching for previous month data.")
    previous_month_str = previous_year_month.strftime("%Y-%m")
    latest_data = df_company_info[df_company_info['날짜'] == previous_month_str]
    cumulative_data = df_company_info[(pd.to_datetime(df_company_info['날짜']) >= pd.to_datetime(f"{latest_year}-01")) & 
                                      (pd.to_datetime(df_company_info['날짜']) < pd.to_datetime(latest_year_month))]

    # 최신 데이터를 바탕으로 '연말 예상' 데이터 계산
    if not latest_data.empty and not cumulative_data.empty:
        # 누적 총자산과 매출액을 이용하여 연환산 계산
        cumulative_total_assets = cumulative_data['총자산'].sum()
        # 총자산의 월평균 계산
        total_assets = int(round((cumulative_total_assets / (latest_month - 1))))  
        # 누적 매출액을 월수로 나누고 12를 곱해 연환산
        cumulative_revenue = cumulative_data['매출액'].sum()
        revenue = int(round((cumulative_revenue / (latest_month - 1)) * 12))

        # 총자산증가율 계산 (직전 연도 12월 대비)
        prev_year_december_date = f"{latest_year - 1}-12"
        prev_december_data = df_company_info[df_company_info['날짜'] == prev_year_december_date]
        if not prev_december_data.empty:
            prev_total_assets = prev_december_data.at[prev_december_data.index[0], '총자산']
            asset_growth_rate = round(((total_assets - prev_total_assets) / prev_total_assets) * 100, 2)
        else:
            asset_growth_rate = 0

        # 매출액증가율 계산 (전년도 누적 매출액 대비)
        prev_annual_data = df_company_info[df_company_info['날짜'].str.startswith(f"{latest_year - 1}")]
        if not prev_annual_data.empty:
            prev_revenue = prev_annual_data['매출액'].sum()
            if prev_revenue != 0:
                revenue_growth_rate = round(((revenue - prev_revenue) / prev_revenue) * 100, 2)
            else:
                revenue_growth_rate = 0  # 이전 매출액이 0인 경우 증가율을 0으로 설정
        else:
            revenue_growth_rate = 0
        
        # 데이터 추가
        growth_data['year_level_data'][estimate_key] = {
            '총자산': total_assets,
            '매출액': revenue
        }
        growth_data['year_rate_data'][estimate_key] = {
            '총자산증가율': asset_growth_rate,
            '매출액증가율': revenue_growth_rate
        }
    else:
        # 데이터가 없는 경우 기본 값 추가 (0)
        growth_data['year_level_data'][estimate_key] = {
            '총자산': 0,
            '매출액': 0
        }
        growth_data['year_rate_data'][estimate_key] = {
            '총자산증가율': 0,
            '매출액증가율': 0
        }

    # 'recent_data'에 대해 최신 년월 기준으로 지난 12개월 데이터를 추출
    past_12_months = pd.date_range(end=latest_year_month, periods=12, freq='ME').strftime("%Y-%m").tolist()
    for month in past_12_months:
        # 해당 월의 데이터를 필터링
        monthly_data = df_company_info[df_company_info['날짜'] == month]
        prev_year_month = (pd.to_datetime(month) - pd.DateOffset(years=1)).strftime("%Y-%m")
        prev_monthly_data = df_company_info[df_company_info['날짜'] == prev_year_month]
        recent_12_months_data = df_company_info[(pd.to_datetime(df_company_info['날짜']) <= pd.to_datetime(month)) & 
                                                (pd.to_datetime(df_company_info['날짜']) > (pd.to_datetime(month) - pd.DateOffset(months=12)))]
        prev_recent_12_months_data = df_company_info[(pd.to_datetime(df_company_info['날짜']) <= pd.to_datetime(prev_year_month)) & 
                                                     (pd.to_datetime(df_company_info['날짜']) > (pd.to_datetime(prev_year_month) - pd.DateOffset(months=12)))]

        if not monthly_data.empty:
            # '성장성' 지표 추출
            revenue = int(round(monthly_data.at[monthly_data.index[0], '매출액']))
            prev_revenue = int(round(prev_monthly_data.at[prev_monthly_data.index[0], '매출액'])) if not prev_monthly_data.empty else 0
            revenue_growth_rate = round(((revenue - prev_revenue) / prev_revenue) * 100, 2) if prev_revenue != 0 else 0
            recent_12_months_revenue = int(round(recent_12_months_data['매출액'].sum()))
            prev_12_months_revenue = int(round(prev_recent_12_months_data['매출액'].sum()))
            cumulative_growth_rate = round(((recent_12_months_revenue - prev_12_months_revenue) / prev_12_months_revenue) * 100, 2) if prev_12_months_revenue != 0 else 0
            
            growth_data['recent_data'][month] = {
                '총자산': monthly_data.at[monthly_data.index[0], '총자산'],
                '매출액': revenue,
                '전년동월 매출액': prev_revenue,
                '매출액증가율': revenue_growth_rate,
                '최근12개월 누적 매출액': recent_12_months_revenue,
                '전년동월 누적 매출액': prev_12_months_revenue,
                '누적 매출액증가율': cumulative_growth_rate
            }
        else:
            # 빈 데이터일 경우 기본 값 추가 (0)
            growth_data['recent_data'][month] = {
                '총자산': 0,
                '매출액': 0,
                '전년동월 매출액': 0,
                '매출액증가율': 0,
                '최근12개월 누적 매출액': 0,
                '전년동월 누적 매출액': 0,
                '누적 매출액증가율': 0
            }
    
    return growth_data


pprint(preprocess_growth_data(df_company_info[df_company_info['기업명'] == 'Company_1']))

In [ ]:
# 재현성을 위한 Random seed 설정
np.random.seed(42)

def preprocess_profitability_data(df_company_info):
    """
    수익성 지표 데이터를 전처리하여 필요한 JSON 형식의 구조로 변환하는 함수.
    
    Args:
        df_company_info (DataFrame): 회사 재무 정보 데이터프레임
    
    Returns:
        dict: 전처리된 수익성 지표 데이터 (JSON 형태)
    """
    # 최신 년월 추출: 데이터프레임에서 가장 최신의 '날짜' 값을 가져옴
    latest_year_month = df_company_info['날짜'].max()
    latest_year = int(latest_year_month.split('-')[0])  # 최신 년도 추출
    latest_month = int(latest_year_month.split('-')[1])  # 최신 월 추출

    # 결과를 저장할 데이터 딕셔너리 정의
    profitability_data = {
        'latest_year_month': latest_year_month,  # 최신 년월 기록
        'year_level_data': {},  # 연간 수준 데이터
        'year_rate_data': {},  # 연간 증가율 데이터
        'recent_data': {}  # 최근 12개월 데이터
    }

    # 'year_level_data' 및 'year_rate_data'에 대해 과거 데이터를 추출
    num_years = 3  # 추출할 과거 연도의 수, 필요에 따라 변경 가능
    for year_offset in range(1, num_years + 1):
        target_year = latest_year - year_offset
        year_key = f"{target_year}년"
        annual_data = df_company_info[df_company_info['날짜'].str.startswith(f"{target_year}")]

        # 연간 데이터를 사용할 수 있는 경우
        if not annual_data.empty:
            # '수익성' 지표 계산
            operating_profit = int(round(annual_data['영업이익'].sum()))  # 해당 연도의 전체 영업이익의 합계
            net_profit = int(round(annual_data['당기순이익'].sum()))  # 해당 연도의 전체 당기순이익의 합계
            revenue = annual_data['매출액'].sum()  # 해당 연도의 매출액 합계

            # 영업이익률과 당기순이익률 계산: 매출액이 있어야 계산 가능
            if revenue != 0:
                operating_profit_margin = round((operating_profit / revenue) * 100, 2)
                net_profit_margin = round((net_profit / revenue) * 100, 2)
            else:
                operating_profit_margin = 0
                net_profit_margin = 0

            # 연도별 데이터 딕셔너리에 추가
            profitability_data['year_level_data'][year_key] = {
                '영업이익': operating_profit,
                '당기순이익': net_profit
            }
            profitability_data['year_rate_data'][year_key] = {
                '영업이익률': operating_profit_margin,
                '당기순이익률': net_profit_margin
            }
        else:
            # 데이터가 없는 경우 기본 값 추가
            profitability_data['year_level_data'][year_key] = {
                '영업이익': 0,
                '당기순이익': 0
            }
            profitability_data['year_rate_data'][year_key] = {
                '영업이익률': 0,
                '당기순이익률': 0
            }

    # 최신 년월의 데이터도 추가 ('연말 예상'으로 표시, 연환산 계산)
    estimate_key = f"{latest_year}년(E)"  # '연말 예상'을 나타내는 키 추가
    max_iterations = 12  # 최대 반복 횟수 설정
    iteration_count = 0
    previous_year_month = pd.to_datetime(latest_year_month) - pd.DateOffset(months=1)
    while previous_year_month.strftime("%Y-%m") not in df_company_info['날짜'].values:
        previous_year_month -= pd.DateOffset(months=1)
        iteration_count += 1
        if iteration_count >= max_iterations:
            raise ValueError("Exceeded maximum iterations while searching for previous month data.")

    previous_month_str = previous_year_month.strftime("%Y-%m")
    latest_data = df_company_info[df_company_info['날짜'] == previous_month_str]
    cumulative_data = df_company_info[(pd.to_datetime(df_company_info['날짜']) >= pd.to_datetime(f"{latest_year}-01")) & (pd.to_datetime(df_company_info['날짜']) < pd.to_datetime(latest_year_month))]

    if not latest_data.empty and not cumulative_data.empty:
        cumulative_operating_profit = cumulative_data['영업이익'].sum()
        cumulative_net_profit = cumulative_data['당기순이익'].sum()
        revenue = cumulative_data['매출액'].sum()

        # 연환산 값 계산
        if latest_month == 1:
            operating_profit = cumulative_operating_profit * 12
            net_profit = cumulative_net_profit * 12
        else:
            operating_profit = (cumulative_operating_profit / (latest_month - 1)) * 12
            net_profit = (cumulative_net_profit / (latest_month - 1)) * 12

        if revenue != 0:
            operating_profit_margin = round((operating_profit / revenue) * 100, 2)
            net_profit_margin = round((net_profit / revenue) * 100, 2)
        else:
            operating_profit_margin = 0
            net_profit_margin = 0

        # 데이터 추가
        profitability_data['year_level_data'][estimate_key] = {
            '영업이익': int(round(operating_profit)),
            '당기순이익': int(round(net_profit))
        }
        profitability_data['year_rate_data'][estimate_key] = {
            '영업이익률': operating_profit_margin,
            '당기순이익률': net_profit_margin
        }
    else:
        profitability_data['year_level_data'][estimate_key] = {
            '영업이익': 0,
            '당기순이익': 0
        }
        profitability_data['year_rate_data'][estimate_key] = {
            '영업이익률': 0,
            '당기순이익률': 0
        }

    # 'recent_data'에 대해 최신 년월 기준으로 지난 12개월 데이터를 추출
    past_12_months = pd.date_range(end=latest_year_month, periods=12, freq='ME').strftime("%Y-%m").tolist()
    for month in past_12_months:
        monthly_data = df_company_info[df_company_info['날짜'] == month]
        prev_year_month = (pd.to_datetime(month) - pd.DateOffset(years=1)).strftime("%Y-%m")
        prev_monthly_data = df_company_info[df_company_info['날짜'] == prev_year_month]
        recent_12_months_data = df_company_info[(pd.to_datetime(df_company_info['날짜']) <= pd.to_datetime(month)) & (pd.to_datetime(df_company_info['날짜']) > (pd.to_datetime(month) - pd.DateOffset(months=12)))]
        prev_recent_12_months_data = df_company_info[(pd.to_datetime(df_company_info['날짜']) <= pd.to_datetime(prev_year_month)) & (pd.to_datetime(df_company_info['날짜']) > (pd.to_datetime(prev_year_month) - pd.DateOffset(months=12)))]

        if not monthly_data.empty:
            operating_profit = int(round(monthly_data.at[monthly_data.index[0], '영업이익']))
            prev_operating_profit = int(round(prev_monthly_data.at[prev_monthly_data.index[0], '영업이익'])) if not prev_monthly_data.empty else 0
            operating_profit_growth_rate = round(((operating_profit - prev_operating_profit) / prev_operating_profit) * 100, 2) if prev_operating_profit != 0 else 0
            recent_12_months_operating_profit = int(round(recent_12_months_data['영업이익'].sum()))
            prev_12_months_operating_profit = int(round(prev_recent_12_months_data['영업이익'].sum()))
            cumulative_growth_rate = round(((recent_12_months_operating_profit - prev_12_months_operating_profit) / prev_12_months_operating_profit) * 100, 2) if prev_12_months_operating_profit != 0 else 0

            profitability_data['recent_data'][month] = {
                '영업이익': operating_profit,
                '전년동월 영업이익': prev_operating_profit,
                '영업이익증가율': operating_profit_growth_rate,
                '최근12개월 누적 영업이익': recent_12_months_operating_profit,
                '전년동월 누적 영업이익': prev_12_months_operating_profit,
                '누적 영업이익증가율': cumulative_growth_rate
            }
        else:
            profitability_data['recent_data'][month] = {
                '영업이익': 0,
                '전년동월 영업이익': 0,
                '영업이익증가율': 0,
                '최근12개월 누적 영업이익': 0,
                '전년동월 누적 영업이익': 0,
                '누적 영업이익증가율': 0
            }

    return profitability_data

pprint(preprocess_profitability_data(df_company_info[df_company_info['기업명'] == 'Company_1']))

## 4. Langchain을 활용한 분석

- cache 설정 : 비용 절감 목적
- streaming을 위한 custom handler 설정
- common_llm : 데이터 기반 해석만 하는 자유도가 낮은 gpt-4o-mini
- load_template : template 읽기
- determine_strength_weakness : **BPI 계산법 통해 연계 필요**
- create_analysis_chain : 분석 chain 생성
- run_analysis : 강/약점을 활용해 분석 시작

In [55]:
# SQLite 캐시 설정 - 반복된 요청에 대한 비용 절감 목적
try:
    langchain_cache = SQLiteCache(database_path="cache.sqlite")
except Exception as e:
    print(f"Error initializing cache: {e}") 
    langchain_cache = None # 캐시 초기화 오류 시 None으로 설정하여 코드가 계속 실행되도록 설정

# 커스텀 콜백 핸들러 정의 - 스트리밍 데이터를 실시간으로 출력하는 handler
class CustomStreamingHandler(BaseCallbackHandler):
    def stream_response(self, token: str, **kwargs) -> None:
        print(token, end="", flush=True) # 토큰 단위로 출력, 실시간 스트리밍 효과 제공
    
# OpenAI GPT-4 Mini 모델 설정 (LLM 설정) - 스트리밍 기능 활성화
common_llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    max_tokens=300,
    streaming=True,  # 스트리밍 활성화
    # cache=langchain_cache # SQLite 캐시 사용하여 비용 절감
    callbacks=[StreamingStdOutCallbackHandler()]
)

# 디렉토리 설정 - 현재 디렉토리와 프롬프트 파일 경로 설정
CURRENT_DIR = os.getcwd()
PROMPT_PATH = os.path.join(CURRENT_DIR, "prompts")

# prompts 폴더에서 prompt txt 파일 읽기 - prompt template을 load하는 함수
def load_prompt(file_name: str) -> str:
    """
    주어진 파일 이름의 프롬프트 템플릿을 읽어 반환합니다.

    Args:
        file_name (str): 로드할 프롬프트 파일의 이름

    Returns:
        str: 파일에서 읽은 프롬프트 템플릿 내용. 파일이 없을 경우 빈 문자열 반환

    Raises:
        FileNotFoundError: 지정된 파일을 찾을 수 없을 때 발생하며, 오류 메시지 출력 후 빈 문자열 반환
    """
    file_path = os.path.join(PROMPT_PATH, file_name)
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: Prompt file {file_name} not found.") # 파일이 없을 경우 오류 메시지 출력
        return ""

# 강점과 약점을 결정하는 함수 - Radar 차트를 이용하여 강점과 약점을 선택하는 로직 (구현 생략)
def determine_strength_weakness(data: pd.DataFrame) -> Dict[str, str]:
    """
    주어진 데이터를 분석하여 기업의 강점과 약점을 결정합니다.

    Args:
        data (pd.DataFrame): 분석할 기업 데이터

    Returns:
        Dict[str, str]: 'strength'와 'weakness' 키를 가진 딕셔너리.
                        각각 강점과 약점으로 판단된 지표명을 값으로 가짐

    Note:
        현재 구현은 예시이며, 실제로는 Radar 차트 등을 이용한 복잡한 로직이 필요함
    """
    # 아래는 예시로 강점과 약점을 임의로 설정함
    strength = 'growth'
    weakness = 'profitability'
    return {'strength': strength, 'weakness': weakness}

# 분석 체인을 생성하는 함수 - 프롬프트와 LLM을 사용해 분석 체인 생성
def create_analysis_chain(indicator: str, is_strength: bool, llm: ChatOpenAI):
    """
    지정된 지표에 대한 분석 체인을 생성합니다.

    Args:
        indicator (str): 분석할 지표명 ('growth' 또는 'profitability')
        is_strength (bool): 강점 분석 여부. True면 강점, False면 약점 분석
        llm (ChatOpenAI): 사용할 언어 모델 인스턴스

    Returns:
        RunnableLambda: 데이터프레임을 입력받아 분석 결과를 반환하는 실행 가능한 람다 함수

    Raises:
        ValueError: 알 수 없는 지표명이 제공될 경우 발생
    """
    # 프롬프트 템플릿 로드 및 추가 분석 지시사항 설정
    base_template = load_prompt(f"{indicator}_template.txt")
    additional_prompt = "Perform analysis on positive side." if is_strength else "Perform analysis on negative side."
    full_template = f"# {base_template}\n\n{additional_prompt}"

    # 프롬프트 템플릿을 사용하여 체인 생성    
    prompt = PromptTemplate.from_template(full_template)
    analysis_chain = prompt | llm | StrOutputParser()

    # 지표에 따른 전처리 함수 설정    
    if indicator == 'growth':
        preprocess_func = preprocess_growth_data
    elif indicator == 'profitability':
        preprocess_func = preprocess_profitability_data
    else:
        raise ValueError(f"Unknown indicator: {indicator}")

    # RunnableLambda를 사용하여 dataframe을 전처리하고 분석하는 lambda 함수 반환    
    return RunnableLambda(lambda df: analysis_chain.invoke({
        "latest_year_month": preprocess_func(df)["latest_year_month"],
        "year_level_data": preprocess_func(df)["year_level_data"],
        "year_rate_data": preprocess_func(df)["year_rate_data"],
        "recent_data": preprocess_func(df)["recent_data"]
    }))

# 분석 체인을 생성하고 결합하는 함수
def run_analysis(df: pd.DataFrame) -> Dict[str, Any]:
    """
    주어진 데이터프레임에 대해 강점과 약점 분석을 수행합니다.

    Args:
        df (pd.DataFrame): 분석할 기업 데이터

    Returns:
        Dict[str, Any]: 다음 키를 포함하는 딕셔너리:
            - 'parallel': 강점과 약점 분석을 병렬로 수행하는 RunnableParallel 객체
            - 'strength': 강점 분석을 위한 RunnableLambda 객체
            - 'weakness': 약점 분석을 위한 RunnableLambda 객체
            - 'strength_name': 강점으로 선택된 지표명
            - 'weakness_name': 약점으로 선택된 지표명

    Note:
        이 함수는 determine_strength_weakness와 create_analysis_chain 함수를 내부적으로 사용합니다.
    """
    # 데이터에 따라 강점과 약점을 결정
    strength_weakness = determine_strength_weakness(df)
    strength = strength_weakness['strength']
    weakness = strength_weakness['weakness']

    # 강점과 약점을 분석하기 위한 체인 생성
    strength_chain = create_analysis_chain(strength, True, common_llm)
    weakness_chain = create_analysis_chain(weakness, False, common_llm)

    # 병렬로 강점과 약점 분석을 수행하는 체인 생성
    parallel_chain = RunnableParallel({
        'strength_analysis': strength_chain,
        'weakness_analysis': weakness_chain
    })

    return {
        'parallel': parallel_chain, 
        'strength': strength_chain, 
        'weakness': weakness_chain,
        'strength_name': strength,
        'weakness_name': weakness
    }

def merge_analysis_results(strength_result: str, weakness_result: str) -> str:
    """
    강점 분석 결과와 약점 분석 결과를 하나의 문자열로 병합합니다.

    Args:
        strength_result (str): 강점 분석 결과 문자열
        weakness_result (str): 약점 분석 결과 문자열

    Returns:
        str: 병합된 분석 결과 문자열
    """
    return '\n\n'.join([
        "Strength Analysis:",
        strength_result,
        "Weakness Analysis:",
        weakness_result
    ])
    
def print_preprocessed_data(df: pd.DataFrame, metric: str) -> None:
    """
    전처리된 데이터를 표 형식으로 출력합니다.

    Args:
        df (pd.DataFrame): 전처리할 데이터프레임
        metric (str): 분석할 지표 ('growth' 또는 'profitability')

    Returns:
        None: 함수는 데이터를 출력만 하고 반환값은 없습니다.

    Note:
        이 함수는 year_level_data, year_rate_data, recent_data를 표 형식으로 출력합니다.
        출력 순서는 2021년, 2022년, 2023년, 2024년(E) 순입니다.
    """
    # 전처리 함수 선택
    preprocess_func = globals()[f"preprocess_{metric}_data"]
    preprocessed_data = preprocess_func(df)
    
    print(f"\n{metric.capitalize()} Data:")

    # year_level_data 출력
    print("\nYear Level Data:")
    year_level_df = pd.DataFrame(preprocessed_data['year_level_data'])
    # 열 순서 변경
    year_level_df = year_level_df[['2021년', '2022년', '2023년', '2024년(E)']]
    print(tabulate(year_level_df, headers='keys', tablefmt='pretty'))
    
    # year_rate_data 출력
    print("\nYear Rate Data:")
    year_rate_df = pd.DataFrame(preprocessed_data['year_rate_data'])
    # 열 순서 변경
    year_rate_df = year_rate_df[['2021년', '2022년', '2023년', '2024년(E)']]
    print(tabulate(year_rate_df, headers='keys', tablefmt='pretty'))
    
    # recent_data 출력
    print("\nRecent Data:")
    recent_df = pd.DataFrame(preprocessed_data['recent_data'])
    print(tabulate(recent_df, headers='keys', tablefmt='pretty'))


## 5. 경영제언 Session
- gpt-4o로 고품질 모델 활용
- 창의도를 결정하는 temperature=1.2로 설정
- max_tokens=500으로 조금 더 긴 text 작성하도록 설정
- final_chain : 앞에서 분석한 strength와 weakness를 합쳐서 경영제언을 하도록 chain으로 연결
    - 1안 : 앞에서 streaming으로 분석한 strength_result와 weakness_result를 묶어서 수행 (토큰 비용 아낌)
    - 2안 : End-to-End로 강단점 분석부터 시작해서 경영 제언까지 수행 (토큰 비용 더 발생) 

In [56]:
insight_llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=1.2,
    max_tokens=500,
    streaming=True,
    # cache=langchain_cache,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [57]:
# 단일 회사 정보만을 필터링하여 사용 (예: 첫 번째 회사)
company_list = df_company_info['기업명'].unique()[:1]
for firm in company_list:
    firm_data = df_company_info[df_company_info['기업명'] == firm]

    # 강점과 약점 분석 체인 생성
    chains = run_analysis(firm_data)
    
    # 병렬 체인 및 강점과 약점 체인 추출
    # parallel_chain = chains.get('parallel', None)  # 키가 없으면 None 반환
    strength_chain = chains.get('strength', None)
    weakness_chain =  chains.get('weakness', None)
    strength_name = chains.get('strength_name', 'Unknown')
    weakness_name = chains.get('weakness_name', 'Unknown')
    
    # # 병렬 분석 수행
    # try:
    #     result = parallel_chain.invoke(temp_df) # 병렬로 강점과 약점 분석을 수행
    # except Exception as e:
    #     print(f"Error during parallel analysis: {e}")
    #     continue

    # # 커스텀 스트리밍 핸들러 설정
    # handler = CustomStreamingHandler()

    # print("\n\n", '#'*10, firm, '#'*10)
    # try:
    #     print(f"\n\n{'#'*10} Analyzing Strength ({strength_name}) for {firm} {'#'*10}")
    #     strength_result = ""
    #     for chunk in strength_chain.stream(firm_data):
    #         handler.stream_response(chunk) # 실시간으로 분석 결과 출력
    #         strength_result += chunk

    #     print(f"\n\n{'#'*10} Analyzing Weakness ({weakness_name}) for {firm} {'#'*10}")
    #     weakness_result = ""
    #     for chunk in weakness_chain.stream(firm_data):
    #         handler.stream_response(chunk)
    #         weakness_result += chunk
    
    # except Exception as e:
    #     print(f"Error during analysis: {e}")
    
    # print(merge_analysis_results(strength_result, weakness_result))

    print(f"\n\n{'#'*10} Preprocessing Strength ({strength_name}) Data for {firm} {'#'*10}")
    print_preprocessed_data(firm_data, strength_name)

    print(f"\n\n{'#'*10} Analyzing Strength ({strength_name}) for {firm} {'#'*10}")
    strength_result = strength_chain.invoke(firm_data)
    
    print(f"\n\n{'#'*10} Preprocessing Weakness ({weakness_name}) Data for {firm} {'#'*10}")
    print_preprocessed_data(firm_data, weakness_name)
    
    print(f"\n\n{'#'*10} Analyzing Weakness ({weakness_name}) for {firm} {'#'*10}")
    weakness_result = weakness_chain.invoke(firm_data)
    
    
    print("\n\n",'#'*10, f'Final Comment for {firm}', '#'*10)    
    final_chain = (
        RunnableParallel({
            'strength_result': itemgetter('strength'),
            'weakness_result': itemgetter('weakness')
        })
        | RunnableLambda(lambda x: {'info': merge_analysis_results(x['strength_result'], x['weakness_result'])})
        | PromptTemplate.from_template(
            """
        Translate and analyze the provided company's strengths and weaknesses:

        {info}

        - Analyze the company's strengths and weaknesses based on the provided data, focusing on key business management insights.
        - Provide specific and actionable recommendations, grounded in the analysis, to capitalize on strengths and address weaknesses.
        - Highlight critical trends and figures as evidence to support your recommendations.
        - Ensure that the recommendations are concise (limited to 5 lines) and translated into Korean in a professional tone.

        # Output Format
        - The response should be in Korean, using natural and connected language.
        - Limit the response to 5 lines or less, with a clear and cohesive structure.
        - Write in a continuous paragraph format that flows smoothly between data points, avoiding lists or bullet points.

        # Guidelines
        1. Base your response objectively on the given data without speculation.
        2. Use specific figures and trends to support your analysis.
        3. Provide practical recommendations that are directly actionable.
        4. Maintain clarity and professionalism in both the analysis and the translation to Korean.
            """
        ) 
        | insight_llm 
    )
    
    result = final_chain.invoke({
    'strength': strength_result,
    'weakness': weakness_result
    })
    print('\n')



########## Preprocessing Strength (growth) Data for Company_1 ##########

Growth Data:

Year Level Data:
+--------+--------+--------+--------+-----------+
|        | 2021년 | 2022년 | 2023년 | 2024년(E) |
+--------+--------+--------+--------+-----------+
| 총자산 | 30687  | 29873  | 29656  |   29669   |
| 매출액 | 72399  | 76678  | 54363  |   61275   |
+--------+--------+--------+--------+-----------+

Year Rate Data:
+--------------+--------+--------+--------+-----------+
|              | 2021년 | 2022년 | 2023년 | 2024년(E) |
+--------------+--------+--------+--------+-----------+
| 총자산증가율 | -5.99  | -2.65  | -0.73  |   0.04    |
| 매출액증가율 | 21.16  |  5.91  | -29.1  |   12.71   |
+--------------+--------+--------+--------+-----------+

Recent Data:
+------------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|                        | 2023-09 | 2023-10 | 2023-11 | 2023-12 | 2024-01 | 2024-02 | 2024-03 | 2024-04 

In [45]:


print(tabulate(pd.DataFrame(preprocess_growth_data(firm_data)['year_level_data']), headers = 'keys', tablefmt='psql'))

# pd.DataFrame(preprocess_growth_data(firm_data)['year_level_data'])

+--------+----------+----------+----------+-------------+
|        |   2023년 |   2022년 |   2021년 |   2024년(E) |
|--------+----------+----------+----------+-------------|
| 총자산 |    29656 |    29873 |    30687 |       29669 |
| 매출액 |    54363 |    76678 |    72399 |       61275 |
+--------+----------+----------+----------+-------------+


In [ ]:
# run_analysis 함수의 결과를 사용하여 final_chain 구성
def create_final_chain(df: pd.DataFrame):
    chains = run_analysis(df)
    parallel_chain = chains['parallel']

    final_chain = (
        parallel_chain
        | RunnableLambda(lambda x: {
            'info': merge_analysis_results(
                x['strength_analysis'],
                x['weakness_analysis']
            ),
            'strength_name': chains['strength_name'],
            'weakness_name': chains['weakness_name']
        })
        | PromptTemplate.from_template(
            """
            Translate and analyze the provided company's strengths ({strength_name}) and weaknesses ({weakness_name}):

            {info}

            - Analyze the company's strengths and weaknesses based on the provided data, focusing on key business management insights.
            - Provide specific and actionable recommendations, grounded in the analysis, to capitalize on strengths and address weaknesses.
            - Highlight critical trends and figures as evidence to support your recommendations.
            - Ensure that the recommendations are concise (limited to 5 lines) and translated into Korean in a professional tone.

            # Output Format
            - The response should be in Korean.
            - Limit the response to 5 lines or less, with a clear and cohesive structure.
            - Write in a continuous paragraph format that flows smoothly between data points, avoiding lists or bullet points.

            # Guidelines
            1. Base your response objectively on the given data without speculation.
            2. Use specific figures and trends to support your analysis.
            3. Provide practical recommendations that are directly actionable.
            4. Maintain clarity and professionalism in both the analysis and the translation to Korean.
            """
        ) 
        | insight_llm 
        | StrOutputParser()
    )

    return final_chain

# 실행

# 단일 회사 정보만을 필터링하여 사용 (예: 첫 번째 회사)
company_list = df_company_info['기업명'].unique()[:3]
for firm in company_list:
    firm_data = df_company_info[df_company_info['기업명'] == firm]
    final_chain = create_final_chain(firm_data)
    print("\n", '#'*10, f'Final Comment for {firm}', '#'*10)
    result = final_chain.invoke(firm_data)
    print('\n')


In [ ]:
print(result)